**Preparación de datos**

*Importación de paquetes y librerías utilizadas en el ejercicio*



In [ ]:
import numpy as np
import pandas as pd
import numbers
import statsmodels.api as sm
import scipy.stats as stats
from joblib import dump, load

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

import scipy.stats as stats


In [ ]:
df = pd.read_csv("./gapminder_taller.csv", delimiter=",")

In [ ]:
df.shape

In [ ]:
df.dtypes

In [ ]:
df.sample(5)

*Calidad de datos: encontrar valores faltantes en la base*

In [ ]:
for col in df.columns:

    num_na = df[col].isna().sum()
    num_non_num = df[col].apply(lambda x: not isinstance(x, numbers.Number)).sum()
    print(f"Columna: {col}")
    print(f"Número de valores vacíos: {num_na}")
    print(f"Número de valores no numéricos: {num_non_num}")

*Encontrar valores duplicados en la base*

In [ ]:
# Elimina los valores duplicados de todas las columnas
df_wd = df.drop_duplicates()

In [ ]:
df_wd.shape

In [ ]:
df_wd.dtypes

*Encontrar valores NaN en la base de datos*

In [ ]:
# Crea un diccionario con el número de datos NaN por columna
nan_values_by_column = df_wd.isna().sum().to_dict()

# Imprime el resumen de cada columna
for column, nan_values in nan_values_by_column.items():
    print(f"Columna: {column} | Número de datos NaN: {nan_values}")

*Seleccionar el dataframe con 5 variables para trabajar. Por fidelidad de los datos se buscará trabajar con las más completas para poder evitar la mayor cantidad de imputación de datos, así como los datos faltantes*

In [ ]:
new_df = df_wd[["country", "incomeperperson", "alcconsumption", "breastcancerper100th","armedforcesrate","lifeexpectancy", "suicideper100th", "urbanrate", "employrate"]]


In [ ]:
new_df.loc[df['country'] == 'Afghanistan', 'incomeperperson'] = 1824
new_df.loc[df['country'] == 'Myanmar', 'incomeperperson'] = 3851
new_df.loc[df['country'] == 'Kuwait', 'incomeperperson'] = 52920
new_df.loc[df['country'] == 'Djibouti', 'employrate'] = 24.068

In [ ]:
# Crea un diccionario con el número de datos NaN por columna
nan_values_by_column = new_df.isna().sum().to_dict()

# Imprime el resumen de cada columna
for column, nan_values in nan_values_by_column.items():
    print(f"Columna: {column} | Número de datos NaN: {nan_values}")

In [ ]:
new_df.loc[new_df['country'] == 'Timor-Leste', 'breastcancerper100th'] = 27.4

In [ ]:
# Usar indexación booleana para encontrar los países con NaN en "hdi_2010"
paises_con_nan = new_df[new_df['breastcancerper100th'].isna()]['country']

# Esto creará una serie con los nombres de los países que tienen NaN en "hdi_2010"
print("Países con NaN en la columna 'breastcancerper100th':")
print(paises_con_nan)

In [ ]:
new_df.loc[new_df['country'] == "Swaziland", 'armedforcesrate'] = 0
new_df.loc[new_df['country'] == "Solomon Islands", 'armedforcesrate'] = 0
new_df.loc[new_df['country'] == "Bhutan", 'armedforcesrate'] = 0
new_df.loc[new_df['country'] == "Comoros", 'armedforcesrate'] = 0

In [ ]:
# Usar indexación booleana para encontrar los países con NaN en "hdi_2010"
paises_con_nan = new_df[new_df['armedforcesrate'].isna()]['country']

# Esto creará una serie con los nombres de los países que tienen NaN en "hdi_2010"
print("Países con NaN en la columna 'armedforcesrate':")
print(paises_con_nan)

*Complemento del dataset con Indice de Desarrollo Humano y Regiones de cada país*

In [ ]:
idh = pd.read_csv("./IDH.csv", delimiter=";")

In [ ]:
idh.shape

In [ ]:
idh.dtypes


In [ ]:
new_df3 = new_df.merge(idh, how="left", on="country")

In [ ]:
new_df3.loc[new_df3['country'] == 'Central African Rep.', 'hdi_2010'] = 0.404
new_df3.loc[new_df3['country'] == 'Macedonia, FYR', 'hdi_2010'] = 0.770
new_df3.loc[new_df3['country'] == 'Korea, Rep.', 'hdi_2010'] = 0.925
new_df3.loc[new_df3['country'] == 'Cape Verde', 'hdi_2010'] = 0.662
new_df3.loc[new_df3['country'] == 'Yemen, Rep.', 'hdi_2010'] = 0.455
new_df3.loc[new_df3['country'] == 'Congo, Dem. Rep.', 'hdi_2010'] = 0.479
new_df3.loc[new_df3['country'] == 'Congo, Rep.', 'hdi_2010'] = 0.571
new_df3.loc[new_df3['country'] == 'Dominican Rep.', 'hdi_2010'] = 0.767
new_df3.loc[new_df3['country'] == 'Czech Rep.', 'hdi_2010'] = 0.889

In [ ]:
new_df3.sample(166)

In [ ]:
new_df3.dtypes

In [ ]:
new_df3.shape

In [ ]:
# Usar indexación booleana para encontrar los países con NaN en "region"
paises_con_nan = new_df3[new_df3['region'].isna()]['country']

# Esto creará una serie con los nombres de los países que tienen NaN en "region"
print("Países con NaN en la columna 'region':")
print(paises_con_nan)

In [ ]:
new_df3.loc[new_df3['country'] == 'Central African Rep.', 'region'] = "SSA"
new_df3.loc[new_df3['country'] == 'Macedonia, FYR', 'region'] = "ECA"
new_df3.loc[new_df3['country'] == 'Korea, Rep.', 'region'] = "EAP"
new_df3.loc[new_df3['country'] == 'Cape Verde', 'region'] = "SSA"
new_df3.loc[new_df3['country'] == 'Yemen, Rep.', 'region'] = "AS"
new_df3.loc[new_df3['country'] == 'Congo, Dem. Rep.', 'region'] = "SSA"
new_df3.loc[new_df3['country'] == 'Congo, Rep.', 'region'] = "SSA"
new_df3.loc[new_df3['country'] == 'Dominican Rep.', 'region'] = "LAC"
new_df3.loc[new_df3['country'] == 'Czech Rep.', 'region'] = "ECA"

In [ ]:
new_df3 = new_df3.drop(columns=['hdicode'])

In [ ]:
new_df3.sample(166)

In [ ]:
new_df4 = new_df3[["country", "incomeperperson", "alcconsumption", "lifeexpectancy","breastcancerper100th", "armedforcesrate","suicideper100th", "urbanrate", "employrate","region","hdi_2010"]]

In [ ]:
new_df4.sample(10)

In [ ]:
new_df4.shape

In [ ]:
new_df4.dtypes

In [ ]:
# Calcula la matriz de correlación
correlation_matrix = new_df4.corr()

# Visualiza la matriz de correlación usando seaborn
plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1)
plt.title('Matriz de Correlación')
plt.show()

In [ ]:
# Aplicar una transformación logarítmica a la variable "incomeperperson"
new_df4['incomeperperson_log'] = np.log(new_df4['incomeperperson'])

# Visualizar el DataFrame con la nueva columna logarítmica
print(new_df4[['country', 'incomeperperson', 'incomeperperson_log']])

In [ ]:
nan_values = new_df4.isna()
cantidad_nan = nan_values.sum().sum()
print("Cantidad de valores NaN en el DataFrame:", cantidad_nan)

In [ ]:
# Crea un diccionario con el número de datos NaN por columna
nan_values_by_column = new_df4.isna().sum().to_dict()

# Imprime el resumen de cada columna
for column, nan_values in nan_values_by_column.items():
    print(f"Columna: {column} | Número de datos NaN: {nan_values}")

**Análisis Univariado**

In [ ]:
# Variables que deseas excluir
excluir_columnas = ["country", "region"]

# Genera un histograma para cada variable excluyendo "country" y "region"
for column in new_df4.columns:
    if column not in excluir_columnas:
        fig, ax = plt.subplots(figsize=(12, 6))
        new_df4[column].hist(ax=ax, bins=20)
        ax.set_title(column)
        plt.show()

**Haciendo la regresión lineal**

In [ ]:
# Crear un modelo de regresión lineal
modelo = LinearRegression()

In [ ]:
y = new_df4["incomeperperson_log"].values.reshape(-1, 1)

In [ ]:
X = new_df4[["alcconsumption", "lifeexpectancy","breastcancerper100th", "armedforcesrate","suicideper100th", "urbanrate", "employrate","hdi_2010"]].values

In [ ]:
X.shape

In [ ]:
y.shape

In [ ]:
# Añadir una constante a X (esto añade la columna para el término de intercepto)
X_constante = sm.add_constant(X)

# Ajustar el modelo
modelo_statsmodels = sm.OLS(y, X_constante).fit()

# Imprimir el resumen
print(modelo_statsmodels.summary())


**Regresión lineal ajustada por parámetros**


In [ ]:
y2 = new_df4["incomeperperson_log"].values.reshape(-1, 1)

In [ ]:
X2 = new_df4[["alcconsumption","armedforcesrate","urbanrate","hdi_2010"]].values

In [ ]:
# Añadir una constante a X (esto añade la columna para el término de intercepto)
X2_constante = sm.add_constant(X2)
modelo_statsmodels = sm.OLS(y2, X2_constante).fit()
print(modelo_statsmodels.summary())